In [1]:
import numpy as np
import sklearn
from sklearn import datasets
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

### The aim of this assignment is to predict auto insurance pricing using ML algorithms

In [10]:
data_train_insurance = pd.read_csv('data_train_insurance.csv',sep=';')

In [11]:
data_train_insurance.head()

,Id,Response_Market_Index,PROD_Formula,PER_sex,PER_Sex_Second_Driver,VEH_Actual_Value,CLA_HISTORY,CLA_HISTORY_1Y,CLA_HISTORY_3Y,CLA_BM_TPL_DECLARED,...,VEH_pw_proportion,VEH_High_Risk_Vehicle,PER_previous_insurer,PER_Is_second_driver,PER_Age_oldest_child,PER_Second_Driver_Age,PER_Insured_Before,VEH_Weight,GEO_Lat,GEO_Long
0,1638099,435.0,BASIC,2,0,4600,1,1,1,1,...,59,2,5,1,-99,-1,2,1125,-1.790876,-0.431147
1,1657342,609.0,FULL,1,0,68315,1,1,1,2,...,106,2,17,1,2,-1,2,1510,1.675824,-4.431147
2,1657529,609.0,FULL,1,0,68315,1,1,1,2,...,106,2,17,1,2,-1,2,1510,1.675824,-4.431147
3,1662071,404.0,FULL,1,2,11005,1,1,1,2,...,51,2,6,2,1,31,2,1170,-0.440876,1.285553
4,1662074,404.0,FULL,1,2,11005,1,1,1,2,...,51,2,6,2,1,31,2,1170,-0.440876,1.285553


In [17]:
data_train_insurance.dtypes

Id                                 int64
Response_Market_Index            float64
PROD_Formula                      object
PER_sex                            int64
PER_Sex_Second_Driver              int64
VEH_Actual_Value                   int64
CLA_HISTORY                       object
CLA_HISTORY_1Y                    object
CLA_HISTORY_3Y                    object
CLA_BM_TPL_DECLARED               object
BEH_Mileage                       object
PER_KIDS_ON_BOARD_ID              object
VEH_Number_of_doors               object
VEH_Top_Speed                     object
BEH_Number_Year_Insured           object
VEH_TransmissionId                object
PER_Age_Main_Driver               object
PER_License_Age_Main_Driver       object
PER_License_Age_Second_Driver     object
PER_Matrimonial_Status            object
VEH_Age                           object
VEH_Injection_Mode                object
VEH_fuel_type                     object
BEH_vehicule_usage                object
VEH_Drive_type  